In [1]:
import pandas as pd
import numpy as np
import pickle
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
import string
import re
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as VS
# from textstat.textstat import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn import cross_validation
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.models import model_from_json
from keras.optimizers import Adam

/home/bhargav/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
embeddings_index = dict()
f = open('../glove.6B/glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [3]:
df = pd.read_csv("../Proper Experiments/Parsed Datasets/StormfrontWS.csv")
df.text=df.text.astype(str)
df.head(5)

,Unnamed: 0,class,text
0,0,0,march 13th 2014 booklet downloaded 18 300 time...
1,1,0,order help increase booklets downloads would g...
2,2,0,simply copy paste following text youtube video...
3,3,1,click free download colorfully illustrated 132...
4,4,0,click download 7 42 green banner link


In [4]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
vocabulary_size=len(tokenizer.word_index) + 1
print(vocabulary_size)

embedding_matrix = np.zeros((vocabulary_size, 300))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

15690


In [5]:
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [6]:
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    break

In [7]:
hidden_size=[75,100,125]
learning_rate=[0.0001, 0.001, 0.01]
score=[]
accuracy=[]

for i in learning_rate:
    sco=[]
    acc=[]
    for j in hidden_size:
        
        model_glove = Sequential()
        model_glove.add(Embedding(vocabulary_size, 300, input_length=80, weights=[embedding_matrix], trainable=False))
        model_glove.add(LSTM(300))
        model_glove.add(Dense(j, activation='relu'))
        model_glove.add(Dense(1, activation='sigmoid'))
        model_glove.compile(loss='binary_crossentropy', optimizer=Adam(lr=i), metrics=['accuracy'])
        
        model_glove.fit(Xtrain, np.array(ytrain), epochs = 3)

        pred=model_glove.predict_classes(X_val)
        
        acc.append(accuracy_score(y_val, pred))
        sco.append(f1_score(y_val, pred, average='macro'))
    score.append(sco)
    accuracy.append(acc)

Epoch 1/3
7704/7704 [==============================] - 45s 6ms/step - loss: 0.3954 - acc: 0.8702
Epoch 2/3
7704/7704 [==============================] - 48s 6ms/step - loss: 0.2692 - acc: 0.8917
Epoch 3/3
7704/7704 [==============================] - 42s 5ms/step - loss: 0.2549 - acc: 0.8972
Epoch 1/3
7704/7704 [==============================] - 58s 8ms/step - loss: 0.3790 - acc: 0.8751
Epoch 2/3
7704/7704 [==============================] - 52s 7ms/step - loss: 0.2674 - acc: 0.8964
Epoch 3/3
7704/7704 [==============================] - 55s 7ms/step - loss: 0.2546 - acc: 0.8988
Epoch 1/3
7704/7704 [==============================] - 56s 7ms/step - loss: 0.3692 - acc: 0.8879
Epoch 2/3
7704/7704 [==============================] - 52s 7ms/step - loss: 0.2684 - acc: 0.8959
Epoch 3/3
7704/7704 [==============================] - 55s 7ms/step - loss: 0.2495 - acc: 0.8990
Epoch 1/3
7704/7704 [==============================] - 54s 7ms/step - loss: 0.3068 - acc: 0.8854
Epoch 2/3
7704/7704 [=========

In [8]:
print(score)

[[0.608236629667003, 0.5832005614669988, 0.5736816097949323], [0.6966786817713697, 0.6197420404369045, 0.6586687306501547], [0.608236629667003, 0.7172650475184794, 0.565317491388816]]


In [10]:
ac=[]
sc=[]
i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    model_glove = Sequential()
    model_glove.add(Embedding(vocabulary_size, 300, input_length=80, weights=[embedding_matrix], trainable=False))
    model_glove.add(LSTM(300))
    model_glove.add(Dense(100, activation='relu'))
    model_glove.add(Dense(1, activation='sigmoid'))
    model_glove.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.01), metrics=['accuracy'])
    
    print("Fold Number: "+str(i))  
    model_glove.fit(Xtrain, np.array(ytrain), epochs = 5)
    
    pred=model_glove.predict_classes(Xtest)
    
    a=accuracy_score(ytest, pred)
    s=f1_score(ytest, pred, average='macro')
    
    if i==0:
        # serialize model to JSON
        model_json = model_glove.to_json()
        with open("Models/model_StormfrontWS.json", "w") as json_file:
            json_file.write(model_json)
        # serialize weights to HDF5
        model_glove.save_weights("Models/model_StormfrontWS.h5")
        print("Saved model to disk")
    elif s>max(sc):
        # serialize model to JSON
        model_json = model_glove.to_json()
        with open("Models/model_StormfrontWS.json", "w") as json_file:
            json_file.write(model_json)
        # serialize weights to HDF5
        model_glove.save_weights("Models/model_StormfrontWS.h5")
        print("Saved model to disk")

    ac.append(a)
    sc.append(s)
    i=i+1    

Fold Number: 0
Epoch 1/5
7704/7704 [==============================] - 41s 5ms/step - loss: 0.3111 - acc: 0.8904
Epoch 2/5
7704/7704 [==============================] - 45s 6ms/step - loss: 0.2526 - acc: 0.8989
Epoch 3/5
7704/7704 [==============================] - 41s 5ms/step - loss: 0.2419 - acc: 0.9072
Epoch 4/5
7704/7704 [==============================] - 42s 5ms/step - loss: 0.2042 - acc: 0.9187
Epoch 5/5
7704/7704 [==============================] - 39s 5ms/step - loss: 0.1571 - acc: 0.9404
Saved model to disk
Fold Number: 1
Epoch 1/5
7706/7706 [==============================] - 43s 6ms/step - loss: 0.3050 - acc: 0.8881
Epoch 2/5
7706/7706 [==============================] - 41s 5ms/step - loss: 0.2626 - acc: 0.8993
Epoch 3/5
7706/7706 [==============================] - 39s 5ms/step - loss: 0.2209 - acc: 0.9136
Epoch 4/5
7706/7706 [==============================] - 35s 5ms/step - loss: 0.1766 - acc: 0.9332
Epoch 5/5
7706/7706 [==============================] - 35s 5ms/step - loss: 0

In [11]:
print("Accuracies: ")
print(ac)
print("Mean: "+str(np.mean(np.array(ac))))
print("Std Dev: "+str(np.std(np.array(ac))))

Accuracies: 
[0.8952282157676349, 0.8670820353063343, 0.8925233644859814, 0.8888888888888888, 0.8655244029075805]
Mean: 0.881849381471284
Std Dev: 0.012861269168092312


In [12]:
print("Macro-F1-Scores: ")
print(sc)
print("Mean: "+str(np.mean(np.array(sc))))
print("Std Dev: "+str(np.std(np.array(sc))))

Macro-F1-Scores: 
[0.6965870562425209, 0.6886706673940716, 0.680470299707866, 0.6769560678310431, 0.6808996124102911]
Mean: 0.6847167407171586
Std Dev: 0.007062134241712507


In [5]:
# load json and create model
json_file = open('Models/model_StormfrontWS.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_glove = model_from_json(loaded_model_json)
# load weights into new model
model_glove.load_weights("Models/model_StormfrontWS.h5")
print("Loaded model from disk")

Loaded model from disk


In [6]:
# Dataset-1
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Davidson.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

   Unnamed: 0                                               text  class
0           0   ! ! ! mayasolovely : woman complain cleaning ...      0
1           1   ! ! ! ! ! mleew17 : boy dats cold tyga dwn ba...      1
2           2   ! ! ! ! ! ! ! urkindofbrand dawg ! ! ! ! 80sb...      1
3           3   ! ! ! ! ! ! ! ! ! c g anderson : viva based l...      1
4           4   ! ! ! ! ! ! ! ! ! ! ! ! ! shenikaroberts : sh...      1


In [7]:
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_StormfrontWS.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    StormfrontWS_model = model_from_json(loaded_model_json)
    # load weights into new model
    StormfrontWS_model.load_weights("Models/model_StormfrontWS.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in StormfrontWS_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in StormfrontWS_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in StormfrontWS_model.layers:
        print(layer, layer.trainable)
    StormfrontWS_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    StormfrontWS_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=StormfrontWS_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f9ad4d67898> False
<keras.layers.recurrent.LSTM object at 0x7f9ad4d679b0> True
<keras.layers.core.Dense object at 0x7f9ad4d67978> True
<keras.layers.core.Dense object at 0x7f9acd5bdcf8> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f9ad4d67898> False
<keras.layers.recurrent.LSTM object at 0x7f9ad4d679b0> False
<keras.layers.core.Dense object at 0x7f9ad4d67978> True
<keras.layers.core.Dense object at 0x7f9acd5bdcf8> True
Epoch 1/8
4460/4460 [==============================] - 9s 2ms/step - loss: 1.0508 - acc: 0.5211
Epoch 2/8
4460/4460 [==============================] - 7s 2ms/step - loss: 0.4088 - acc: 0.8334
Epoch 3/8
4460/4460 [==============================] - 9s 2ms/step - loss: 0.3759 - acc: 0.8370
Epoch 4/8
4460/4460 [==============================] - 9s 2ms/step - loss: 0.3550 - acc: 0.8433
Epoch 5/8
4460/4460 [==============================] - 10s 2ms/step - loss:

In [8]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))

With Fine Tuning: 
Accuracies: 
[0.8431196772747647, 0.8442402510085164, 0.8385650224215246, 0.8381165919282512, 0.8403587443946189]
Mean: 0.8408800574055352
Std Dev: 0.00243206471479882
Macro-F1-Scores: 
[0.619067829684268, 0.6027301039981665, 0.54796177382773, 0.6083791099202388, 0.5947946206724772]
Mean: 0.5945866876205761
Std Dev: 0.024616247814312014


In [9]:
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

Without Fine Tuning: 
Accuracies: 
[0.2008068130883012, 0.20484087852980726, 0.2069506726457399, 0.2031390134529148, 0.20605381165919281]
Mean: 0.2043582378751912
Std Dev: 0.002187590304734578
Macro-F1-Scores: 
[0.19022782643532526, 0.19414730727756657, 0.19563461900587295, 0.19241281220888912, 0.195907416062005]
Mean: 0.1936659961979318
Std Dev: 0.002122384533927916


In [10]:
# Dataset-2
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Waseem_Hovy.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

   Unnamed: 0                                               text  class
0           0   mralmubarak yes tried murder way heaven total...      1
1           1   nigzzzk command fear allah quran ritual grove...      1
2           2   ib alkisai islamic documentation based belief...      1
3           3   benfrancisallen : good weeks isis new front o...      0
4           4  self described radical feminist makes death th...      1


In [11]:
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_StormfrontWS.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    StormfrontWS_model = model_from_json(loaded_model_json)
    # load weights into new model
    StormfrontWS_model.load_weights("Models/model_StormfrontWS.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in StormfrontWS_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in StormfrontWS_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in StormfrontWS_model.layers:
        print(layer, layer.trainable)
    StormfrontWS_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    StormfrontWS_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=StormfrontWS_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f9a5cbfb780> False
<keras.layers.recurrent.LSTM object at 0x7f9acfcb8898> True
<keras.layers.core.Dense object at 0x7f9a611abc88> True
<keras.layers.core.Dense object at 0x7f9acfc76da0> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f9a5cbfb780> False
<keras.layers.recurrent.LSTM object at 0x7f9acfcb8898> False
<keras.layers.core.Dense object at 0x7f9a611abc88> True
<keras.layers.core.Dense object at 0x7f9acfc76da0> True
Epoch 1/8
2841/2841 [==============================] - 6s 2ms/step - loss: 0.6642 - acc: 0.7114
Epoch 2/8
2841/2841 [==============================] - 5s 2ms/step - loss: 0.5658 - acc: 0.7209
Epoch 3/8
2841/2841 [==============================] - 5s 2ms/step - loss: 0.5455 - acc: 0.7385
Epoch 4/8
2841/2841 [==============================] - 5s 2ms/step - loss: 0.5300 - acc: 0.7494
Epoch 5/8
2841/2841 [==============================] - 5s 2ms/step - loss: 

In [12]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))

With Fine Tuning: 
Accuracies: 
[0.7432289834681675, 0.754750175932442, 0.7501759324419423, 0.7543983110485574, 0.7472720872932066]
Mean: 0.7499650980368632
Std Dev: 0.004363996919527351
Macro-F1-Scores: 
[0.6627207260851363, 0.6673100872253895, 0.6575601444553056, 0.6742058945642406, 0.6862401696160383]
Mean: 0.6696074043892221
Std Dev: 0.009956259007385072


In [13]:
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

Without Fine Tuning: 
Accuracies: 
[0.7013717903622934, 0.7146375791695989, 0.7005629838142153, 0.7054890921885996, 0.7092573037662795]
Mean: 0.7062637498601972
Std Dev: 0.005217518146397425
Macro-F1-Scores: 
[0.49707155515225643, 0.5273476999526089, 0.49044052689873185, 0.5048886633587839, 0.5175043177892918]
Mean: 0.5074505526303346
Std Dev: 0.013420897929089367


In [14]:
# Dataset-3
df=pd.read_csv('../Proper Experiments/Parsed Datasets/OffensEval_2019.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

   Unnamed: 0                                               text  class
0           0                 user ask native americans take is       1
1           1   user user home you re drunk ! ! ! user maga t...      1
2           2  amazon investigating chinese employees selling...      0
3           3   user someone should have taken piece shit vol...      1
4           4   user user obama wanted liberals amp; illegals...      0


In [15]:
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_StormfrontWS.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    StormfrontWS_model = model_from_json(loaded_model_json)
    # load weights into new model
    StormfrontWS_model.load_weights("Models/model_StormfrontWS.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in StormfrontWS_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in StormfrontWS_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in StormfrontWS_model.layers:
        print(layer, layer.trainable)
    StormfrontWS_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    StormfrontWS_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=StormfrontWS_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f9a610c0470> False
<keras.layers.recurrent.LSTM object at 0x7f9a610c0588> True
<keras.layers.core.Dense object at 0x7f9a610c0550> True
<keras.layers.core.Dense object at 0x7f9acd832d30> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f9a610c0470> False
<keras.layers.recurrent.LSTM object at 0x7f9a610c0588> False
<keras.layers.core.Dense object at 0x7f9a610c0550> True
<keras.layers.core.Dense object at 0x7f9acd832d30> True
Epoch 1/8
2383/2383 [==============================] - 6s 2ms/step - loss: 0.8370 - acc: 0.6609
Epoch 2/8
2383/2383 [==============================] - 5s 2ms/step - loss: 0.6353 - acc: 0.6614
Epoch 3/8
2383/2383 [==============================] - 6s 3ms/step - loss: 0.6122 - acc: 0.6744
Epoch 4/8
2383/2383 [==============================] - 5s 2ms/step - loss: 0.5936 - acc: 0.6949
Epoch 5/8
2383/2383 [==============================] - 5s 2ms/step - loss: 

In [16]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))

With Fine Tuning: 
Accuracies: 
[0.6954697986577181, 0.687788501888376, 0.6982794796475031, 0.6961812840956777, 0.6966009232060428]
Mean: 0.6948639974990636
Std Dev: 0.0036564273923101426
Macro-F1-Scores: 
[0.5970959352591338, 0.5710918222196417, 0.5948575491603006, 0.5941123551520421, 0.574107166031584]
Mean: 0.5862529655645405
Std Dev: 0.011231736901848784


In [17]:
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

Without Fine Tuning: 
Accuracies: 
[0.6774328859060402, 0.6760386067981536, 0.6718422156945028, 0.6743600503566932, 0.668485102811582]
Mean: 0.6736317723133943
Std Dev: 0.003176867184591729
Macro-F1-Scores: 
[0.4729228737251781, 0.4809286692955864, 0.4724628474628475, 0.4738867797412577, 0.46077448522705916]
Mean: 0.4721951310903858
Std Dev: 0.006483344786046689


In [6]:
# Dataset-4
df=pd.read_csv('../Proper Experiments/Parsed Datasets/hatEval_2019.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

   Unnamed: 0                                               text  class
0           0  hurray saving many ways potus realdonaldtrump ...      0
1           1  would young fighting age men vast majority one...      0
2           2   kamalaharris illegals dump kids border like r...      0
3           3  times : nearly white states pose an array prob...      0
4           4  orban brussels : european leaders ignoring peo...      0


In [7]:
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_StormfrontWS.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    StormfrontWS_model = model_from_json(loaded_model_json)
    # load weights into new model
    StormfrontWS_model.load_weights("Models/model_StormfrontWS.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in StormfrontWS_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in StormfrontWS_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in StormfrontWS_model.layers:
        print(layer, layer.trainable)
    StormfrontWS_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    StormfrontWS_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=StormfrontWS_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f8761b4fd68> False
<keras.layers.recurrent.LSTM object at 0x7f8761b4fe80> True
<keras.layers.core.Dense object at 0x7f876191a550> True
<keras.layers.core.Dense object at 0x7f8753e14f98> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f8761b4fd68> False
<keras.layers.recurrent.LSTM object at 0x7f8761b4fe80> False
<keras.layers.core.Dense object at 0x7f876191a550> True
<keras.layers.core.Dense object at 0x7f8753e14f98> True
Epoch 1/8
1618/1618 [==============================] - 4s 2ms/step - loss: 0.5598 - acc: 0.7645
Epoch 2/8
1618/1618 [==============================] - 3s 2ms/step - loss: 0.4830 - acc: 0.8028
Epoch 3/8
1618/1618 [==============================] - 3s 2ms/step - loss: 0.4367 - acc: 0.8152
Epoch 4/8
1618/1618 [==============================] - 3s 2ms/step - loss: 0.4162 - acc: 0.8239
Epoch 5/8
1618/1618 [==============================] - 3s 2ms/step - loss: 

In [8]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))

With Fine Tuning: 
Accuracies: 
[0.8245830759728228, 0.818912237330037, 0.818912237330037, 0.8257107540173053, 0.8288009888751545]
Mean: 0.8233838587050715
Std Dev: 0.0039035537289070932
Macro-F1-Scores: 
[0.5381422670802463, 0.5518823203579089, 0.5147472319831969, 0.5292571301229381, 0.5236619445870385]
Mean: 0.5315381788262658
Std Dev: 0.01270174965263274


In [9]:
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

Without Fine Tuning: 
Accuracies: 
[0.7393452748610253, 0.7391841779975278, 0.7181705809641533, 0.7503090234857849, 0.7478368355995055]
Mean: 0.7389691785815994
Std Dev: 0.011313679856270308
Macro-F1-Scores: 
[0.5603691463458579, 0.5531404696895525, 0.5210531782301621, 0.5587971662454913, 0.569141604010025]
Mean: 0.5525003129042177
Std Dev: 0.016539852183861902


In [22]:
# Dataset-5
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Davidson_1.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

   Unnamed: 0                                               text  class
0           0   ! ! ! mayasolovely : woman complain cleaning ...      0
1           1   ! ! ! ! ! mleew17 : boy dats cold tyga dwn ba...      0
2           2   ! ! ! ! ! ! ! urkindofbrand dawg ! ! ! ! 80sb...      0
3           3   ! ! ! ! ! ! ! ! ! c g anderson : viva based l...      0
4           4   ! ! ! ! ! ! ! ! ! ! ! ! ! shenikaroberts : sh...      0


In [23]:
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_StormfrontWS.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    StormfrontWS_model = model_from_json(loaded_model_json)
    # load weights into new model
    StormfrontWS_model.load_weights("Models/model_StormfrontWS.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in StormfrontWS_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in StormfrontWS_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in StormfrontWS_model.layers:
        print(layer, layer.trainable)
    StormfrontWS_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    StormfrontWS_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=StormfrontWS_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f9a3b3ee0f0> False
<keras.layers.recurrent.LSTM object at 0x7f9a3b3eed68> True
<keras.layers.core.Dense object at 0x7f9a3b3ee8d0> True
<keras.layers.core.Dense object at 0x7f9a393a5da0> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f9a3b3ee0f0> False
<keras.layers.recurrent.LSTM object at 0x7f9a3b3eed68> False
<keras.layers.core.Dense object at 0x7f9a3b3ee8d0> True
<keras.layers.core.Dense object at 0x7f9a393a5da0> True
Epoch 1/8
4460/4460 [==============================] - 9s 2ms/step - loss: 0.2569 - acc: 0.9211
Epoch 2/8
4460/4460 [==============================] - 8s 2ms/step - loss: 0.2250 - acc: 0.9365
Epoch 3/8
4460/4460 [==============================] - 8s 2ms/step - loss: 0.2061 - acc: 0.9413
Epoch 4/8
4460/4460 [==============================] - 7s 2ms/step - loss: 0.1941 - acc: 0.9426
Epoch 5/8
4460/4460 [==============================] - 8s 2ms/step - loss: 

In [24]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))

With Fine Tuning: 
Accuracies: 
[0.9424025100851636, 0.9417301658449126, 0.9405829596412556, 0.9434977578475336, 0.9410313901345292]
Mean: 0.9418489567106789
Std Dev: 0.0010305572748093112
Macro-F1-Scores: 
[0.5373869572566613, 0.4888087126185974, 0.522979443194839, 0.5568215693418569, 0.5359516970096662]
Mean: 0.5283896758843242
Std Dev: 0.022549638897201005


In [25]:
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

Without Fine Tuning: 
Accuracies: 
[0.9009412819363514, 0.9043030031376065, 0.9033632286995515, 0.9049327354260089, 0.9031390134529148]
Mean: 0.9033358525304866
Std Dev: 0.0013607996683019361
Macro-F1-Scores: 
[0.5369878399101492, 0.5326778027879794, 0.5226939265430457, 0.5474144533284319, 0.544061814041056]
Mean: 0.5367671673221325
Std Dev: 0.008733891169052693


In [6]:
# Dataset-6
df=pd.read_csv('../Proper Experiments/Parsed Datasets/GAB.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_StormfrontWS.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    StormfrontWS_model = model_from_json(loaded_model_json)
    # load weights into new model
    StormfrontWS_model.load_weights("Models/model_StormfrontWS.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in StormfrontWS_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in StormfrontWS_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in StormfrontWS_model.layers:
        print(layer, layer.trainable)
    StormfrontWS_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    StormfrontWS_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=StormfrontWS_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0  class                                               text
0           0      1  joined gab remind retarded jew haters are typi...
1           1      0       left really scared of https : redd it 9rfkts
2           2      0   that literally looks like monkey supposed pre...
3           3      1                                          dumb cunt
4           4      0                                     makes asshole 
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f9bcd5295f8> False
<keras.layers.recurrent.LSTM object at 0x7f9bcd529748> True
<keras.layers.core.Dense object at 0x7f9bcd5299b0> True
<keras.layers.core.Dense object at 0x7f9bcd2d8e80> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f9bcd5295f8> False
<keras.layers.recurrent.LSTM object at 0x7f9bcd529748> False
<keras.layers.core.Dense object at 0x7f9bcd5299b0> True
<keras.layers.core.Dense object at 0x7f9bcd2d8e80> True
Epoch 1/8
6079/6079 [======

In [7]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.6933070218714027, 0.6958881578947368, 0.6971541371936174, 0.6876130942589241, 0.6808685639085376]
Mean: 0.6909661950254437
Std Dev: 0.006019987641343954
Macro-F1-Scores: 
[0.684102725828837, 0.677197783462606, 0.6824055219030993, 0.6714581213603146, 0.6755959572219721]
Mean: 0.6781520219553657
Std Dev: 0.004599032106735186
Without Fine Tuning: 
Accuracies: 
[0.613057062983062, 0.6208881578947368, 0.6170422766902451, 0.622964303339365, 0.6180292811317651]
Mean: 0.6183962164078348
Std Dev: 0.0033926846047670142
Macro-F1-Scores: 
[0.5322120662773239, 0.5460252444093847, 0.5399116749763972, 0.5455495845287346, 0.5410974696285198]
Mean: 0.540959207964072
Std Dev: 0.0049872880728269505


In [8]:
# Dataset-7
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Reddit.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_StormfrontWS.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    StormfrontWS_model = model_from_json(loaded_model_json)
    # load weights into new model
    StormfrontWS_model.load_weights("Models/model_StormfrontWS.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in StormfrontWS_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in StormfrontWS_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in StormfrontWS_model.layers:
        print(layer, layer.trainable)
    StormfrontWS_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    StormfrontWS_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=StormfrontWS_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0  class                                               text
0           0      1  subsection retarded hungarians ohh boy brace l...
1           1      0   hiii got work 444 mainly typa guys imagine wr...
2           2      0                    wow guess soyboys every country
3           3      0   owen benjamin soyboy song goes every country ...
4           4      0   y all hear sumn means live small town for wor...
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f9b880f7438> False
<keras.layers.recurrent.LSTM object at 0x7f9b880f71d0> True
<keras.layers.core.Dense object at 0x7f9b880f7470> True
<keras.layers.core.Dense object at 0x7f9bcd60be10> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f9b880f7438> False
<keras.layers.recurrent.LSTM object at 0x7f9b880f71d0> False
<keras.layers.core.Dense object at 0x7f9b880f7470> True
<keras.layers.core.Dense object at 0x7f9bcd60be10> True
Epoch 1/8
4018/4018 [======

In [9]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.783777058969893, 0.7799900447984072, 0.7834743653558984, 0.7832254853160776, 0.7906918865107019]
Mean: 0.7842317681901957
Std Dev: 0.003507659356281872
Macro-F1-Scores: 
[0.590259624996935, 0.5745734250322792, 0.5703161875767015, 0.5590178499537366, 0.5955377147772801]
Mean: 0.5779409604673865
Std Dev: 0.013333842852586425
Without Fine Tuning: 
Accuracies: 
[0.728041801443145, 0.7369337979094077, 0.7314584370333499, 0.7292185166749626, 0.732702837232454]
Mean: 0.7316710780586639
Std Dev: 0.0030981500776603643
Macro-F1-Scores: 
[0.5433346241511141, 0.5521320104699443, 0.55200896928176, 0.5432100468243808, 0.5557171254937183]
Mean: 0.5492805552441835
Std Dev: 0.005083495759427462


In [6]:
# Dataset-8
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Davidson_2.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_StormfrontWS.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    StormfrontWS_model = model_from_json(loaded_model_json)
    # load weights into new model
    StormfrontWS_model.load_weights("Models/model_StormfrontWS.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in StormfrontWS_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in StormfrontWS_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in StormfrontWS_model.layers:
        print(layer, layer.trainable)
    StormfrontWS_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    StormfrontWS_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=StormfrontWS_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0  class                                               text
0           0      0   ! ! ! mayasolovely : woman complain cleaning ...
1           1      0              momma said pussy cats inside doghouse
2           2      0   addicted2guys : - simplyaddictedtoguys http :...
3           3      0   allaboutmanfeet : http : t co 3gzupfumev woof...
4           4      0   allyhaaaaa : lemmie eat oreo amp; dishes one ...
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7f3e43fe2048> False
<keras.layers.recurrent.LSTM object at 0x7f3e43fe2630> True
<keras.layers.core.Dense object at 0x7f3e43fe2438> True
<keras.layers.core.Dense object at 0x7f3e43f55da0> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7f3e43fe2048> False
<keras.layers.recurrent.LSTM object at 0x7f3e43fe2630> False
<keras.layers.core.Dense object at 0x7f3e43fe2438> True
<keras.layers.core.Dense object at 0x7f3e43f55da0> True
Epoch 1/8
1006/1006 [======

In [7]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.7767857142857143, 0.7944389275074478, 0.794234592445328, 0.7922465208747514, 0.7972166998011928]
Mean: 0.7909844909828869
Std Dev: 0.007273715654567253
Macro-F1-Scores: 
[0.6523898199875854, 0.6608517388651617, 0.6635326470697536, 0.6630042618531288, 0.6768329554043839]
Mean: 0.6633222846360027
Std Dev: 0.007852464519502986
Without Fine Tuning: 
Accuracies: 
[0.7380952380952381, 0.7408142999006951, 0.7395626242544732, 0.7266401590457257, 0.7435387673956262]
Mean: 0.7377302177383517
Std Dev: 0.005826339626425239
Macro-F1-Scores: 
[0.5193619234770547, 0.5211968848203307, 0.5156529089639458, 0.49033825769927286, 0.5277820230115932]
Mean: 0.5148663995944395
Std Dev: 0.012879025182410877
